<a href="https://colab.research.google.com/github/blowmeaway1234/Machine-Learning/blob/main/CHAPTER_9_Dimensionality_Reduction_Using_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9.1 Reducing Features Using Principal Components

**Problem**
Given a set of features, you want to reduce the number of features while retaining the
variance in the data.

**Solution**
Use principal component analysis with scikit’s PCA:

In [1]:
# Load libraries
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import datasets
# Load the data
digits = datasets.load_digits()
# Standardize the feature matrix
features = StandardScaler().fit_transform(digits.data) #->Chuẩn hóa dữ liệu
# Create a PCA that will retain 99% of variance
pca = PCA(n_components=0.99, whiten=True) #-> thuật toán PCA
# Conduct PCA
features_pca = pca.fit_transform(features)
# Show results
print("Original number of features:", features.shape[1]) # Ban đầu là 64
print("Reduced number of features:", features_pca.shape[1]) # với thuật toán PCA giảm đc 15 % 
# PCA giảm chiều dữ liệu phi tuyến tính

Original number of features: 64
Reduced number of features: 54


# 9.2 Reducing Features When Data Is Linearly Inseparable

**Problem**

You suspect you have linearly inseparable data and want to reduce the dimensions.

**Solution**

Use an extension of principal component analysis that uses kernels to allow for nonlinear dimensionality reduction:

In [2]:
# Load libraries
from sklearn.decomposition import PCA, KernelPCA
from sklearn.datasets import make_circles
# Create linearly inseparable data
features, _ = make_circles(n_samples=1000, random_state=1, noise=0.1, factor=0.1)
# Apply kernal PCA with radius basis function (RBF) kernel
kpca = KernelPCA(kernel="rbf", gamma=15, n_components=1)
features_kpca = kpca.fit_transform(features)
print("Original number of features:", features.shape[1])
print("Reduced number of features:", features_kpca.shape[1])

Original number of features: 2
Reduced number of features: 1


# 9.3 Reducing Features by Maximizing Class Separability

**Problem**

You want to reduce the features to be used by a classifier.

**Solution**

Try linear discriminant analysis (LDA) to project the features onto component axes
that maximize the separation of classes:

In [17]:
# Load libraries
from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# Load Iris flower dataset:
iris = datasets.load_iris()
features = iris.data
target = iris.target
# Create and run an LDA, then use it to transform the features
lda = LinearDiscriminantAnalysis(n_components=1)
features_lda = lda.fit(features, target).transform(features)
# Print the number of features
print("Original number of features:", features.shape[1])
print("Reduced number of features:", features_lda.shape[1])

Original number of features: 4
Reduced number of features: 1


In [18]:
lda.explained_variance_ratio_

array([0.9912126])

Specifically, we can run LinearDiscriminantAnalysis with n_components set to

None to return the ratio of variance explained by every component feature, then cal‐
culate how many components are required to get above some threshold of variance 

explained (often 0.95 or 0.99):


In [23]:
from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# Create and run LDA
lda = LinearDiscriminantAnalysis(n_components=None)
features_lda = lda.fit(features, target)
# Create array of explained variance ratios
lda_var_ratios = lda.explained_variance_ratio_
# Create function
def select_n_components(var_ratio, goal_var: float) -> int:
  # Set initial variance explained so far
  total_variance = 0.0
  # Set initial number of features
  n_components = 0
  # For the explained variance of each feature:
  for explained_variance in var_ratio:
    # Add the explained variance to the total
    total_variance += explained_variance
    # Add one to the number of components
    n_components += 1
    # If we reach our goal level of explained variance
    if total_variance >= goal_var:
      # End the loop
      break
  # Return the number of components
  return n_components
# Run function
select_n_components(lda_var_ratios, 0.95)

1

# 9.4 Reducing Features Using Matrix Factorization

**Problem**

You have a feature matrix of nonnegative values and want to reduce the dimensional‐
ity.

**Solution**

Use non-negative matrix factorization (NMF) to reduce the dimensionality of the fea‐
ture matrix:

In [24]:
# Load libraries
from sklearn.decomposition import NMF
from sklearn import datasets
# Load the data
digits = datasets.load_digits()
# Load feature matrix
features = digits.data
# Create, fit, and apply NMF
nmf = NMF(n_components=10, random_state=1)
features_nmf = nmf.fit_transform(features)
# Show results
print("Original number of features:", features.shape[1])
print("Reduced number of features:", features_nmf.shape[1])

Original number of features: 64
Reduced number of features: 10


/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


# 9.5 Reducing Features on Sparse Data

**Problem**

You have a sparse feature matrix and want to reduce the dimensionality.

**Solution**
Use Truncated Singular Value Decomposition (TSVD):

In [26]:
# Load libraries
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn import datasets
import numpy as np
# Load the data
digits = datasets.load_digits()
# Standardize feature matrix
features = StandardScaler().fit_transform(digits.data)
# Make sparse matrix
features_sparse = csr_matrix(features)
# Create a TSVD
tsvd = TruncatedSVD(n_components=10)
# Conduct TSVD on sparse matrix
features_sparse_tsvd = tsvd.fit(features_sparse).transform(features_sparse)
# Show results
print("Original number of features:", features_sparse.shape[1])
print("Reduced number of features:", features_sparse_tsvd.shape[1])

Original number of features: 64
Reduced number of features: 10


TSVD is similar to PCA and in fact, PCA actually often uses non-truncated Singular
Value Decomposition (SVD) in one of its steps. In regular SVD, given d features,
SVD will create factor matrices that are d × d, whereas TSVD will return factors that
are n × n, where n is previously specified by a parameter. The practical advantage of
TSVD is that unlike PCA, it works on sparse feature matrices.

In [27]:
# Sum of first three components' explained variance ratios
tsvd.explained_variance_ratio_[0:3].sum()

0.30039385392832774

We can automate the process by creating a function that runs TSVD with n_compo
nents set to one less than the number of original features and then calculate the num‐
ber of components that explain a desired amount of the original data’s variance:

In [29]:
# Create and run an TSVD with one less than number of features
tsvd = TruncatedSVD(n_components=features_sparse.shape[1]-1)
features_tsvd = tsvd.fit(features)
# List of explained variances
tsvd_var_ratios = tsvd.explained_variance_ratio_
# Create a function
def select_n_components(var_ratio, goal_var):
  # Set initial variance explained so far
  total_variance = 0.0
  # Set initial number of features
  n_components = 0
  # For the explained variance of each feature:
  for explained_variance in var_ratio:
    # Add the explained variance to the total
    total_variance += explained_variance
    # Add one to the number of components
    n_components += 1
    # If we reach our goal level of explained variance
    if total_variance >= goal_var:
      # End the loop
      break
  # Return the number of components
  return n_components
# Run function
select_n_components(tsvd_var_ratios, 0.95)

40